In [39]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer 

In [2]:
df=pd.read_csv('train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.shape

(20800, 5)

In [5]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
###Drop Nan Values
df=df.dropna()


In [7]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
df.shape

(18285, 5)

In [9]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [10]:
## Get the Dependent features
y=df['label']

In [11]:
X.shape

(18285, 4)

In [12]:
y.shape

(18285,)

In [13]:
tf.__version__

'2.10.1'

In [14]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [15]:
messages=X.copy()

In [16]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [17]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [18]:
messages.reset_index(inplace=True)

In [19]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [22]:
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [23]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [24]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [25]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[4990, 63, 359, 807, 408, 4426, 4923, 3950, 35, 4859],
 [1997, 1100, 4510, 4592, 1230, 875, 1500],
 [4556, 4443, 437, 1379],
 [2818, 1195, 1511, 2840, 954, 1646],
 [2943, 1230, 4589, 1884, 2071, 3267, 1230, 3658, 198, 2410],
 [4188,
  3987,
  3727,
  4613,
  3751,
  2509,
  3487,
  1609,
  4564,
  4666,
  1901,
  212,
  1159,
  4457,
  1500],
 [673, 1805, 943, 4672, 3874, 2994, 526, 3508, 234, 1305, 569],
 [1890, 3066, 831, 4239, 4084, 322, 2509, 2908, 234, 1305, 569],
 [2490, 147, 1297, 907, 2855, 3702, 2098, 25, 2509, 298],
 [4916, 2484, 4237, 3966, 2644, 92, 1395, 1418],
 [4189, 2620, 945, 3411, 878, 3845, 757, 4096, 237, 2159, 2406],
 [2840, 4800, 408, 3702, 2509, 4084],
 [2420, 1865, 3854, 3181, 1750, 3186, 4817, 4844, 901],
 [2875, 4593, 3964, 4445, 593, 3093, 292, 234, 1305, 569],
 [965, 697, 3265, 266, 779, 234, 1305, 569],
 [2140, 3652, 4981, 1560, 2354, 2742, 2355, 1931, 3309, 3909],
 [1740, 1322, 1100],
 [4484, 886, 3747, 476, 2509, 494, 3063, 1500],
 [2524, 1877, 4510, 140

In [26]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [27]:
onehot_repr[1]

[1997, 1100, 4510, 4592, 1230, 875, 1500]

### Embedding Representation

In [28]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[4990   63  359 ...    0    0    0]
 [1997 1100 4510 ...    0    0    0]
 [4556 4443  437 ...    0    0    0]
 ...
 [ 994 2664 1923 ...    0    0    0]
 [4643 4084 1604 ...    0    0    0]
 [3493  735  456 ...    0    0    0]]


In [29]:
embedded_docs[1]

array([1997, 1100, 4510, 4592, 1230,  875, 1500,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0])

In [30]:
embedded_docs[0]

array([4990,   63,  359,  807,  408, 4426, 4923, 3950,   35, 4859,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0])

In [31]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [32]:
len(embedded_docs),y.shape

(18285, (18285,))

In [34]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [35]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [38]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 10s 10ms/step - loss: 0.3183 - accuracy: 0.8437 - val_loss: 0.2063 - val_accuracy: 0.9171
Epoch 2/10
192/192 [==============================] - 2s 8ms/step - loss: 0.1416 - accuracy: 0.9469 - val_loss: 0.2145 - val_accuracy: 0.9171
Epoch 3/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0913 - accuracy: 0.9686 - val_loss: 0.2701 - val_accuracy: 0.9125
Epoch 4/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0579 - accuracy: 0.9799 - val_loss: 0.2719 - val_accuracy: 0.9097
Epoch 5/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0417 - accuracy: 0.9871 - val_loss: 0.3746 - val_accuracy: 0.9114
Epoch 6/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0315 - accuracy: 0.9902 - val_loss: 0.3736 - val_accuracy: 0.9100
Epoch 7/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0259 - accuracy: 0.9914 - val_loss: 0.4449 - val_accuracy: 0.9017
Epoc

### Adding Dropout

In [ ]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [40]:
y_pred=model.predict(X_test)

189/189 [==============================] - 1s 2ms/step


In [41]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [42]:
from sklearn.metrics import confusion_matrix

In [43]:
confusion_matrix(y_test,y_pred)

array([[3087,  332],
       [ 224, 2392]], dtype=int64)

In [44]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.907870753935377

In [45]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.90      0.92      3419
           1       0.88      0.91      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

